In [99]:
from __future__ import absolute_import, division, print_function, unicode_literals

from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

import os
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D, Reshape, GlobalAveragePooling1D
 


##### Data preprocessing

In [75]:
def feature_normalize(dataset):

    mu = np.mean(dataset)
    sigma = np.std(dataset)
    
    return (dataset - mu)/sigma

In [68]:
def get_class_data(class_name):
    global class_counts
    
    path = os.path.join(train_path, class_name)
    class_units = np.fromfile(path, dtype=int, count=-1, sep=' ', offset=0)
    units = np.split(class_units, class_units.size // 1080, 0) # 1080 = 3 sec * 360Hz
    class_counts[class_name] = class_units.size // 1080
    
    return units

In [103]:
base_path = 'C:\\Users\\User\\Desktop\\1studing\\Курсач 3 курс\\course_work'
train_path = os.path.join(base_path, 'data_files')

classes = ['N', 'LBBB', 'RBBB', 'B', 'T', 'VT', 'SVTA', 'AFIB', 'AFL', 'NOD', 'IVR', 'V']
class_mapping = {'N': 0, 'LBBB': 1, 'RBBB': 2, 'B': 3, 'T': 4, 'VT': 5, 'SVTA': 6, 'AFIB': 7, 'AFL': 8, 'NOD': 9, 'IVR': 10, 'V': 11}
class_counts = {'N': 0, 'LBBB': 0, 'RBBB': 0, 'B': 0, 'T': 0, 'VT': 0, 'SVTA': 0, 'AFIB': 0, 'AFL': 0, 'NOD': 0, 'IVR': 0, 'V': 0}
X_train = [] # data - each unit 3 sec - 1080 values
Y_train = [] #labels - from class_mapping

all_data = [] # 12 lists of lists for each class
for class_i in classes:
    class_units = get_class_data(class_i)
    all_data.append(class_units)
    Y_train  += [class_mapping[class_i]] * class_counts[class_i]
    #class_units = feature_normalize(class_units)  # ??
    X_train += class_units
    
#Y_train = to_categorial(Y_train)

#print(len(all_data)) #12
#print(len(X_train)) #8079
#print(len(Y_train)) #8079
#print(X_train)


##### Build the model

In [130]:
TIME_PERIODS = 1080

#model = Sequential()
#model.add(Conv1D(32, 3, 
#          activation='relu', 
#          input_shape=(8079, 1080)))

num_classes = 12
#input_shape = 1080 # ??

model = Sequential()
model.add(Reshape((TIME_PERIODS, 1), input_shape=(input_shape,)))  # ??
#model.add(Conv1D(100, 10, activation='relu', input_shape=(8079, 1080)))
model.add(Conv1D(100, 10, activation='relu', input_shape=(TIME_PERIODS, 1))) # n_timesteps,n_features ?
#model.add(Conv1D(100, 10, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(160, 10, activation='relu'))
#model.add(Conv1D(160, 10, activation='relu'))
#model.add(GlobalAveragePooling1D())
model.add(MaxPooling1D(3))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

print(model.summary())

model_save_path = os.path.join(base_path, 'model\\cnn_model.h5') # or \\ ?
model.save(model_save_path)


Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_12 (Reshape)         (None, 1080, 1)           0         
_________________________________________________________________
conv1d_63 (Conv1D)           (None, 1071, 100)         1100      
_________________________________________________________________
max_pooling1d_19 (MaxPooling (None, 357, 100)          0         
_________________________________________________________________
conv1d_64 (Conv1D)           (None, 348, 160)          160160    
_________________________________________________________________
max_pooling1d_20 (MaxPooling (None, 116, 160)          0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 116, 160)          0         
_________________________________________________________________
dense_15 (Dense)             (None, 116, 12)         

##### Training

In [131]:
output_weight_path = os.path.join(base_path, 'model\\cnn_model_weights_2.hdf5') # or \\ ?

X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)
X_train = X_train.astype("float32")
Y_train = Y_train.astype("float32")

print("\n--- Fit the model ---\n")

# The EarlyStopping callback monitors training accuracy:
# if it fails to improve for two consecutive epochs,
# training stops early
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='accuracy', patience=1)
]

model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

# Hyper-parameters
BATCH_SIZE = 100
EPOCHS = 10 # more ??

# Enable validation to use ModelCheckpoint and EarlyStopping callbacks.
history = model.fit(X_train,
                      Y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      #callbacks=callbacks_list,     
                      validation_split=0.2,
                      verbose=1) 

#model.load_weights(output_weight_path, by_name=True) # later - loading pretrained weights ? for testing
model.save_weights(output_weight_path)

# %%

print(history.history.keys())

print("\n--- Learning curve of model training ---\n")

# summarize history for accuracy and loss
plt.figure(figsize=(20, 10)) # ??
plt.plot(history.history['accuracy'], "g--", label="Accuracy of training data")
plt.plot(history.history['val_accuracy'], "g", label="Accuracy of validation data")
plt.plot(history.history['loss'], "r--", label="Loss of training data")
plt.plot(history.history['val_loss'], "r", label="Loss of validation data")
plt.title('Model Accuracy and Loss')
plt.ylabel('Accuracy and Loss')
plt.xlabel('Training Epoch')
plt.ylim(0)
plt.legend()
plt.show()


--- Fit the model ---

Train on 6463 samples, validate on 1616 samples
Epoch 1/10
 100/6463 [..............................] - ETA: 1:32

InvalidArgumentError:  assertion failed: [] [Condition x == y did not hold element-wise:] [x (loss/dense_15_loss/SparseSoftmaxCrossEntropyWithLogits/Shape_1:0) = ] [100 1] [y (loss/dense_15_loss/SparseSoftmaxCrossEntropyWithLogits/strided_slice:0) = ] [100 116]
	 [[node loss/dense_15_loss/SparseSoftmaxCrossEntropyWithLogits/assert_equal/Assert/Assert (defined at C:\Users\User\Anaconda3\envs\course_work\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_distributed_function_22899]

Function call stack:
distributed_function


##### Testing